In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn3, venn3_circles
from difflib import SequenceMatcher as sm
from utils.merge import merge

In [2]:
sea_lst = []
for y in range(1990, 2024):
    for s in ['03-31', '06-30', '09-30', '12-31']:
        sea_lst.append(f'{y}-{s}')
sea_df = pd.DataFrame(sea_lst[:-3], columns=['会计期间'])
sea_df['on'] = 0

stk_df1 = pd.read_csv('../../data/csmar/公司文件.csv')
stk_df1['on'] = 0
header1 = pd.merge(stk_df1, sea_df, on=['on'], how='left')[['证券代码', '会计期间']]

stk_df2 = pd.read_csv('../../data/tushare/股票列表.csv')
stk_df2['on'] = 0
header2 = pd.merge(stk_df2, sea_df, on=['on'], how='left')[['证券代码', '会计期间']]

stk_df3 = pd.read_csv('../../data/jqdata/所有股票数据.csv')
stk_df3['on'] = 0
header3 = pd.merge(stk_df3, sea_df, on=['on'], how='left')[['证券代码', '会计期间']]

header = pd.merge(pd.merge(header1, header2, on=['证券代码', '会计期间'], how='outer'), header3, on=['证券代码', '会计期间'], how='outer')
header = header.sort_values(by=['证券代码', '会计期间'])
header

,证券代码,会计期间
0,1,1990-03-31
1,1,1990-06-30
2,1,1990-09-30
3,1,1990-12-31
4,1,1991-03-31
...,...,...
728835,900957,2022-03-31
728836,900957,2022-06-30
728837,900957,2022-09-30
728838,900957,2022-12-31


In [3]:
df1 = pd.read_csv('../../data/csmar/季_资产负债表.csv', low_memory=False).rename(columns={'开发支出': '研发支出', '应收分保合同准备金净额': '应收分保合同准备金', '买入返售金融资产净额': '买入返售金融资产', '可供出售金融资产净额': '可供出售金融资产', '生产性生物资产净额': '生产性生物资产', '持有至到期投资净额': '持有至到期投资', '应收分保账款净额': '应收分保账款', '长期股权投资净额': '长期股权投资', '投资性房地产净额': '投资性房地产', '长期应收款净额': '长期应收款', '其他应收款净额': '其他应收款', '预付款项净额': '预付款项', '在建工程净额': '在建工程', '应收账款净额': '应收账款', '应收保费净额': '应收保费', '应收利息净额': '应收利息', '应收股利净额': '应收股利', '应收票据净额': '应收票据', '固定资产净额': '固定资产', '拆出资金净额': '拆出资金', '无形资产净额': '无形资产', '油气资产净额': '油气资产', '商誉净额': '商誉', '存货净额': '存货', '发放贷款及垫款净额': '发放贷款及垫款', '保户质押贷款净额': '保户质押贷款', '其中：库存股': '库存股', '所有者权益合计': '股东权益总计', '负债与所有者权益总计': '负债及股东权益', '加：未确认的投资损失': '未确认的投资损失', '其中:长期健康险责任准备金': '长期健康险责任准备金', '其中:应收分保长期健康险责任准备金净额': '应收分保长期健康险责任准备金', '其中:未到期责任准备金': '未到期责任准备金', '其中:应收分保未到期责任准备金净额': '应收分保未到期责任准备金', '其中:寿险责任准备金': '寿险责任准备金', '其中:未决赔款准备金': '未决赔款准备金', '其中:应收分保未决赔款准备金净额': '应收分保未决赔款准备金', '其中:应收分保寿险责任准备金净额': '应收分保寿险责任准备金', '其中:客户资金存款': '客户资金存款', '其中：吸收存款': '吸收存款', '递延收益-流动负债': '递延收益', '其中：同业及其他金融机构存放款项': '同业及其他金融机构存放款项', '其中：永续债': '永续债', '其中：优先股': '优先股', '长期债权投资净额': '长期债权投资', '短期投资净额': '短期投资', '长期投资净额': '长期投资', '应收代位追偿款净额': '应收代位追偿款', '其中：客户备付金': '客户备付金', '其中:交易席位费': '交易席位费', '其中:质押借款': '质押借款', '流动资产合计': '流动资产', '非流动资产合计': '非流动资产', '流动负债合计': '流动负债', '长期负债合计': '长期负债', '非流动负债合计': '非流动负债', '负债合计': '负债总计', '归属于母公司所有者权益合计': '归属于母公司股东权益'}).drop(columns=['其中：其他', '报表类型', '差错更正披露日期', '证券简称']).replace(0, np.nan)
df2 = pd.read_csv('../../data/tushare/季_资产负债表.csv').rename(columns={'资本公积金': '资本公积', '盈余公积金': '盈余公积', '负债及股东权益总计': '负债及股东权益', '减:库存股': '库存股', '股东权益合计(含少数股东权益)': '股东权益总计', '其中：客户资金存款': '客户资金存款', '其他权益工具:永续债': '永续债', '其他权益工具(优先股)': '优先股', '同业和其它金融机构存放款项': '同业及其他金融机构存放款项', '存放同业和其它金融机构款项': '存放同业及其他金融机构款项', '其中：客户备付金': '客户备付金', '其中:交易席位费': '交易席位费', '其中:质押借款': '质押借款', '流动资产合计': '流动资产', '非流动资产合计': '非流动资产', '流动负债合计': '流动负债', '非流动负债合计': '非流动负债', '负债合计': '负债总计', '股东权益合计(不含少数股东权益)': '股东权益(不含少数股东权益)', '期末总股本': '总股本'}).drop(columns=['TS股票代码', '报告期类型', '长期应付款合计', '其他应收款合计', '其他应付款合计', '固定资产合计', '在建工程合计', '待摊费用', '实际公告日期', '报表类型', '公司类型', '公告日期']).replace(0, np.nan)
df3 = pd.read_csv('../../data/jqdata/季_资产负债数据.csv').rename(columns={'开发支出': '研发支出', '资本公积金': '资本公积', '盈余公积金': '盈余公积', '发放委托贷款及垫款': '发放贷款及垫款', '归属于母公司股东权益合计': '归属于母公司股东权益', '负债和股东权益合计': '负债及股东权益', '流动资产合计': '流动资产', '非流动资产合计': '非流动资产', '流动负债合计': '流动负债', '非流动负债合计': '非流动负债', '负债合计': '负债总计', '股东权益合计': '股东权益总计'}).drop(columns=['股票代码', '报告日期']).replace(0, np.nan)

In [4]:
df3 = pd.merge(header, df3, on=['证券代码', '会计期间'], how='left')
df4 = merge(df3, df1, df2, on=['证券代码', '会计期间'])
col = ['应收账款', '其他应付款', '其他应收款', '其他负债', '其他资产']
df4 = pd.merge(df4, df2[['证券代码', '会计期间'] + col], on=['证券代码', '会计期间'], how='left')
for c in col:
    df4[c] = df4[c + '_x'].fillna(df4[c + '_y'])
    df4.drop(columns=[c + '_x', c + '_y'], inplace=True)
col = sorted(list(set(df4.columns) - {'证券代码', '会计期间', '查询成功'}))
df4 = df4[['证券代码', '会计期间', '查询成功'] + col]
df4.to_csv('../../data/dvlab/季_资产负债表.csv', index=False, encoding='utf-8-sig')
df4

,证券代码,会计期间,查询成功,一年内到期的非流动负债,一年内到期的非流动资产,一般风险准备,专项储备,专项应付款,买入返售金融资产,交易席位费,...,长期投资,长期股权投资,长期负债,非流动负债,非流动资产,预付款项,预提费用,预收保费,预收款项,预计负债
0,1,1990-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1990-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1990-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1990-12-31,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1991-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730431,900957,2022-03-31,1.0,70129445.30,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.128559e+08,3.871292e+08,3.871292e+08,7.325872e+08,10990.0,NaN,NaN,NaN,NaN
730432,900957,2022-06-30,1.0,68625793.64,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.154678e+08,3.871292e+08,3.871292e+08,7.265717e+08,102000.0,NaN,NaN,NaN,NaN
730433,900957,2022-09-30,1.0,67911918.89,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.187594e+08,3.871292e+08,3.871292e+08,7.201656e+08,583800.0,NaN,NaN,NaN,NaN
730434,900957,2022-12-31,1.0,75567828.36,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.207923e+08,3.773408e+08,3.826668e+08,7.108852e+08,181160.0,NaN,NaN,NaN,5326000.0
